# Azure AI Agents wit Model Context Protocol (MCP) Support - Python

Dis notebook go show how you fit use Azure AI Agents wit Model Context Protocol (MCP) tools for Python. E go show how you fit create one smart agent wey go fit use external MCP servers (like Microsoft Learn) to get better abilities wit keyless authentication.


## Install Di Python Packages Wey You Need

First, we go need install di Python packages wey dey important:
- **azure-ai-projects**: Di main Azure AI Projects SDK
- **azure-ai-agents**: Azure AI Agents SDK wey you go use take create and manage agents
- **azure-identity**: E dey provide authentication wey no need key, e dey use DefaultAzureCredential
- **mcp**: Na Model Context Protocol implementation for Python


## Benefits of Keyless Authentication

Dis notebook dey show how **keyless authentication** fit help, and e get plenty benefits:
- ✅ **No need to dey manage API keys** - E dey use Azure identity-based authentication
- ✅ **Better security** - No need to store secrets for code or config files
- ✅ **Automatic credential rotation** - Azure go handle di credential lifecycle management
- ✅ **Role-based access control** - E dey use Azure RBAC to give fine-grained permissions
- ✅ **Support for different environments** - E dey work well for both development and production

`DefaultAzureCredential` go automatically pick di best credential source wey dey available:
1. **Managed Identity** (if you dey run am for Azure)
2. **Azure CLI** credentials (if you dey do local development)
3. **Visual Studio** credentials
4. **Environment variables** (if you don configure am)
5. **Interactive browser** authentication (as last option)


## Keyless Authentication Setup

**Wetin you go need for keyless authentication:**

### For Local Development:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### For Azure Environments:
- Turn on **System-assigned Managed Identity** for your Azure resource
- Give di managed identity correct **RBAC roles**:
  - `Cognitive Services OpenAI User` to fit use Azure OpenAI
  - `AI Developer` to fit use Azure AI Projects

### Environment Variables (Optional):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**You no need API keys or connection strings!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Import Di Libraries Wey You Need

Bring in di Python modules wey you go need:
- **os, time**: Na di standard Python libraries wey dey for environment variables and delay
- **AIProjectClient**: Na di main client wey dey for Azure AI Projects
- **DefaultAzureCredential**: Na di way wey no need key to take connect Azure services
- **MCP-related classes**: Na for create and manage MCP tools plus handle approvals


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Set MCP Server Settings

Set up di MCP server config usin environment variables wit fallback defaults:
- **MCP_SERVER_URL**: Di URL of di MCP server (e go default to Microsoft Learn API)
- **MCP_SERVER_LABEL**: One label to identify di MCP server (e go default to "mslearn")

Dis method go make di config flexible for different environments.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Create Azure AI Project Client (Keyless Authentication)

Make Azure AI Project client wey no need key:
- **endpoint**: Na di Azure AI Foundry project endpoint URL
- **credential**: E go use `DefaultAzureCredential()` wey secure, no need key
- **No API keys required**: E go find di best credential wey dey available by itself

**Authentication Flow:**
1. E go first check Managed Identity (for Azure environment)
2. If e no see, e go use Azure CLI credentials (for local development)
3. E go use any other credential wey dey available if e need am

Dis way no go make you dey manage API keys or connection strings for your code.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Create MCP Tool Definition

Make MCP tool wey go connect to Microsoft Learn MCP server:
- **server_label**: Name wey go identify MCP server
- **server_url**: URL endpoint for MCP server
- **allowed_tools**: Optional list wey fit limit which tools agent fit use (empty list go allow all tools)

Dis tool go make agent fit access Microsoft Learn documentation and resources.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Create Agent and Execute Conversation (Keyless Workflow)

Dis kain section go show di full **keyless agent workflow**:

1. **Create AI Agent**: Set up one agent wey go use GPT-4.1 nano model and MCP tools
2. **Create Thread**: Start one conversation thread wey go dey for communication
3. **Send Message**: Ask di agent about di difference between Azure OpenAI and OpenAI
4. **Handle Tool Approvals**: Approve MCP tool calls automatically if dem need am
5. **Monitor Execution**: Dey check how di agent dey progress and handle any action wey dem need
6. **Display Results**: Show di conversation and di tool usage details

**Keyless Features:**
- ✅ **No hardcoded secrets** - All authentication dey handled by Azure identity
- ✅ **Secure by default** - E dey use role-based access control
- ✅ **Simplified deployment** - No need to manage credentials
- ✅ **Audit-friendly** - All access dey tracked through Azure identity

Di agent go use MCP tools to access Microsoft Learn resources with full security and no need to manage API key.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dis dokyument don use AI transleto service [Co-op Translator](https://github.com/Azure/co-op-translator) do di translation. Even as we dey try make am accurate, abeg sabi say machine translation fit get mistake or no dey correct well. Di original dokyument wey dey im native language na di main source wey you go fit trust. For important mata, e good make professional human transleto check am. We no go fit take blame for any misunderstanding or wrong interpretation wey fit happen because you use dis translation.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
